In [11]:
import pandas as pd
DDDB = pd.read_csv("./DDDB_ICD-11.csv", sep=",")
DGIDB = pd.read_csv("../Data/DGIDB/converted/human/dgidb_ncbi_v2.csv")
HUMANNET_PATH = "../Data/HumanNet/HumanNet-GSP.tsv"
HUMANNET = pd.read_csv(HUMANNET_PATH, sep="\t")
DDDB


,NDF-RT,SNOMED,CommonName,Score,ICD11Code,SpecificMatch,BroadName
0,N0000004713,26929004,Alzheimer's disease,94.444444,8,Diseases of the nervous system,Alzheimer disease
1,N0000004713,56267009,Multi-infarct dementia,100.000000,6,"Mental, behavioural or neurodevelopmental diso...",Dementia
2,N0000004713,80098002,Diffuse Lewy body disease,100.000000,8,Diseases of the nervous system,Lewy body disease
3,N0000004713,386806002,Impaired cognition,68.085106,10,Diseases of the ear or mastoid process,Congenital hearing impairment
4,N0000004713,425390006,Dementia associated with Parkinson's Disease,100.000000,6,"Mental, behavioural or neurodevelopmental diso...",Dementia
...,...,...,...,...,...,...,...
3534,N0000146103,19905009,Chronic prostatitis,82.051282,13,Diseases of the digestive system,Chronic pancreatitis
3535,N0000146103,423561003,Community-acquired methicillin-resistant Staph...,64.285714,3,Diseases of the blood or blood-forming organs,Infection of spleen
3536,N0000146103,429271009,Ventilator associated pneumonia,100.000000,12,Diseases of the respiratory system,Pneumonia
3537,N0000146103,192701001,Toxoplasma encephalitis,77.419355,1,Certain infectious or parasitic diseases,Fungal encephalitis


In [12]:
# GET all the drugs for each common name, store as json "COMMON NAME": [Drug1, drug2, ...]
name_grouped = DDDB.groupby('CommonName').agg({
    'NDF-RT': list,
    'ICD11Code': 'first'
}).reset_index()
name_grouped


,CommonName,NDF-RT,ICD11Code
0,AL amyloidosis,"[N0000147066, N0000148588, N0000148811]",5
1,AV junctional rhythm,[N0000148010],11
2,Abdominal aortic aneurysm,"[N0000147924, N0000148200, N0000148821]",11
3,Aberrant premature complexes,[N0000148010],X
4,Abnormal uterine bleeding unrelated to menstru...,[N0000147899],16
...,...,...,...
1245,Waldenström macroglobulinemia,"[N0000146229, N0000146452, N0000146473, N00001...",4
1246,Wolff-Parkinson-White pattern,[N0000147996],8
1247,Wolfram syndrome,[N0000146198],4
1248,Xerostomia,[N0000148658],13


In [13]:
import requests
import re
# Set your BioPortal API key here
BIOPORTAL_API_KEY = "ddb8139c-1d85-4ccf-96c0-2f855b5114ae"

def get_drug_name_from_ndfrt(ndfrt_code):
    """Fetches the drug name for a given NDF-RT code using the BioPortal API."""
    base_url = f"https://data.bioontology.org/ontologies/NDFRT/classes/http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FNDFRT%2F{ndfrt_code}"
    
    headers = {"Authorization": f"apikey token={BIOPORTAL_API_KEY}"}
    
    response = requests.get(base_url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        name = data.get("prefLabel", "Unknown Drug")  # Extract drug name (preferred label)
    else:
        print(f"Error {response.status_code}: {response.text}")
        name = "Unknown Drug"
    name = re.sub(r'\s*\[Chemical/Ingredient\]', '', name).strip()
    
    return name

# Example Usage
ndfrt_code = "N0000146152"
drug_name = get_drug_name_from_ndfrt(ndfrt_code)
print(f"NDF-RT Code: {ndfrt_code} -> Drug Name: {drug_name}")

NDF-RT Code: N0000146152 -> Drug Name: CLONAZEPAM


In [14]:
name_grouped["Drugs"] = name_grouped["NDF-RT"].apply(lambda x: [get_drug_name_from_ndfrt(ndfrt_code).upper() for ndfrt_code in x])
name_grouped.to_csv("Checkpoint.csv", index=False)

Error 404: {"errors":["Resource 'http://purl.bioontology.org/ontology/NDFRT/N0000145804' not found in ontology NDFRT submission 13"],"status":404}
Error 404: {"errors":["Resource 'http://purl.bioontology.org/ontology/NDFRT/N0000145804' not found in ontology NDFRT submission 13"],"status":404}
Error 404: {"errors":["Resource 'http://purl.bioontology.org/ontology/NDFRT/N0000145804' not found in ontology NDFRT submission 13"],"status":404}
Error 404: {"errors":["Resource 'http://purl.bioontology.org/ontology/NDFRT/N0000145804' not found in ontology NDFRT submission 13"],"status":404}
Error 404: {"errors":["Resource 'http://purl.bioontology.org/ontology/NDFRT/N0000145804' not found in ontology NDFRT submission 13"],"status":404}
Error 404: {"errors":["Resource 'http://purl.bioontology.org/ontology/NDFRT/N0000145804' not found in ontology NDFRT submission 13"],"status":404}
Error 404: {"errors":["Resource 'http://purl.bioontology.org/ontology/NDFRT/N0000145804' not found in ontology NDFRT s

In [15]:
from scipy.sparse import dok_matrix, save_npz
# Calculate gene degrees in HumanNet
genes_in_humannet = pd.unique(HUMANNET[['Gene1', 'Gene2']].values.ravel())
gene_to_degree = {gene: 0 for gene in genes_in_humannet}

for _, row in HUMANNET.iterrows():
    gene_to_degree[row["Gene1"]] += 1
    gene_to_degree[row["Gene2"]] += 1

# Add degrees to DGIDB with fallback to 0.01 for missing genes
DGIDB['ncbi_gene_id'] = DGIDB['ncbi_gene_id'].astype(str)
gene_to_degree = {str(gene): degree for gene, degree in gene_to_degree.items()}
DGIDB['degree'] = DGIDB['ncbi_gene_id'].map(gene_to_degree).fillna(0.01)

# Create mappings for vertices and hyperedges
genes = DGIDB['ncbi_gene_id'].unique()
drugs = DGIDB['drug_name'].unique()
gene_to_index = {gene: i for i, gene in enumerate(genes)}
drug_to_index = {drug: i for i, drug in enumerate(drugs)}

In [16]:
print(len(genes))

4774


In [17]:
import os
import re
from scipy.sparse import dok_matrix, save_npz

def sanitize_filename(name):
    """Sanitize the filename by replacing invalid characters."""
    return re.sub(r'[<>:"/\\|?*]', '_', name)

INITIAL_PATH = "./InitialRWGNNHypergraphs/"
for index, row in name_grouped.iterrows():
    # Create directory using ICD11Code
    os.makedirs(INITIAL_PATH + "/" + row["ICD11Code"], exist_ok=True)
    
    # Sanitize the common name to create a valid file name
    sanitized_common_name = sanitize_filename(row["CommonName"])
    OUTPUT_PATH = INITIAL_PATH + "/" + row["ICD11Code"] + "/" + sanitized_common_name + ".npz"
    
    # Initialize a sparse incidence matrix
    incidence_matrix = dok_matrix((len(genes), len(drugs)), dtype=float)
    
    # Populate the incidence matrix with degree as weight
    for _, dgidb_row in DGIDB.iterrows():
        if dgidb_row['drug_name'] in row["Drugs"]:  # Check if the drug is in the bipolar drugs list
            gene_idx = gene_to_index[dgidb_row['ncbi_gene_id']]
            drug_idx = drug_to_index[dgidb_row['drug_name']]
            degree = dgidb_row['degree']
            incidence_matrix[gene_idx, drug_idx] = degree

    # Convert the DOK matrix to CSR format
    csr_matrix = incidence_matrix.tocsr()

    # Save the matrix as .npz file
    save_npz(OUTPUT_PATH, csr_matrix)
    
    # Print confirmation
    print(f"Weighted incidence matrix saved as '{OUTPUT_PATH}'.")


Weighted incidence matrix saved as './InitialRWGNNHypergraphs//5/AL amyloidosis.npz'.
Weighted incidence matrix saved as './InitialRWGNNHypergraphs//11/AV junctional rhythm.npz'.
Weighted incidence matrix saved as './InitialRWGNNHypergraphs//11/Abdominal aortic aneurysm.npz'.
Weighted incidence matrix saved as './InitialRWGNNHypergraphs//X/Aberrant premature complexes.npz'.
Weighted incidence matrix saved as './InitialRWGNNHypergraphs//16/Abnormal uterine bleeding unrelated to menstrual cycle.npz'.
Weighted incidence matrix saved as './InitialRWGNNHypergraphs//6/Absence seizure.npz'.
Weighted incidence matrix saved as './InitialRWGNNHypergraphs//5/Acanthosis nigricans due to type 2 diabetes mellitus.npz'.
Weighted incidence matrix saved as './InitialRWGNNHypergraphs//14/Acne.npz'.
Weighted incidence matrix saved as './InitialRWGNNHypergraphs//X/Acne agminata.npz'.
Weighted incidence matrix saved as './InitialRWGNNHypergraphs//11/Acne detergicans.npz'.
Weighted incidence matrix saved as

In [18]:
#convert from hypergraph to regular graph
import os
import numpy as np
from scipy.sparse import dok_matrix, csr_matrix, save_npz, load_npz
import networkx as nx

# Paths
INITIAL_PATH = "./InitialRWGNNHypergraphs/"
REGULAR_PATH = "./RegularGraphs/"

# Create RegularGraphs path if it doesn't exist
os.makedirs(REGULAR_PATH, exist_ok=True)

# Loop through all categories and files in INITIAL_PATH
for category in os.listdir(INITIAL_PATH):
    category_path = os.path.join(INITIAL_PATH, category)
    
    if os.path.isdir(category_path):  # Check if it's a folder
        # Create a corresponding category folder in REGULAR_PATH
        regular_category_path = os.path.join(REGULAR_PATH, category)
        os.makedirs(regular_category_path, exist_ok=True)

        # Loop through all .npz files in the category
        for file_name in os.listdir(category_path):
            if file_name.endswith(".npz"):
                # Load the hypergraph
                hypergraph_path = os.path.join(category_path, file_name)
                incidence_matrix = load_npz(hypergraph_path)

                # Convert hypergraph to regular graph
                num_genes, num_drugs = incidence_matrix.shape
                G = nx.Graph()

                # Add gene nodes
                for gene_idx in range(num_genes):
                    G.add_node(gene_idx, type="gene")

                # Add edges by projecting to a regular graph
                co_occurrence_matrix = incidence_matrix @ incidence_matrix.T
                co_occurrence_matrix.setdiag(0)  # Remove self-loops

                # Add edges to the regular graph
                gene_indices, gene_connections = co_occurrence_matrix.nonzero()
                for i, j in zip(gene_indices, gene_connections):
                    weight = co_occurrence_matrix[i, j]
                    if i != j and weight > 0:
                        G.add_edge(i, j, weight=weight)

                # Convert the graph to adjacency matrix in CSR format
                adjacency_matrix = nx.to_scipy_sparse_matrix(G, format="csr")

                # Save the regular graph as an .npz file in RegularGraphs
                output_path = os.path.join(regular_category_path, file_name)
                save_npz(output_path, adjacency_matrix)

                # Print confirmation
                print(f"Converted and saved: {output_path}")


Converted and saved: ./RegularGraphs/1\Actinomycotic infection.npz


c:\Users\richa\Desktop\CodingWorkspaces\DGI-Hypergraph\datatestingvenv\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Converted and saved: ./RegularGraphs/1\Active tuberculosis.npz
Converted and saved: ./RegularGraphs/1\Acute cystitis.npz
Converted and saved: ./RegularGraphs/1\Acute pain.npz
Converted and saved: ./RegularGraphs/1\Allergic bronchopulmonary aspergillosis.npz
Converted and saved: ./RegularGraphs/1\Anthrax.npz
Converted and saved: ./RegularGraphs/1\Aphthous ulceration of skin and_or mucous membrane.npz
Converted and saved: ./RegularGraphs/1\Arteriosclerotic vascular disease.npz
Converted and saved: ./RegularGraphs/1\Arthritis of spine.npz
Converted and saved: ./RegularGraphs/1\Arthritis.npz
Converted and saved: ./RegularGraphs/1\Aspergillosis.npz
Converted and saved: ./RegularGraphs/1\Atypical mycobacterial infection.npz
Converted and saved: ./RegularGraphs/1\Babesiosis.npz
Converted and saved: ./RegularGraphs/1\Bacterial endocarditis.npz
Converted and saved: ./RegularGraphs/1\Bacterial infection of eye.npz
Converted and saved: ./RegularGraphs/1\Bacterial meningitis.npz
Converted and save

In [19]:
#Convert from regular graph to RWGNN style graphs with a graph to index
import os
import numpy as np
import pandas as pd
import networkx as nx
from scipy.sparse import load_npz

# Paths
REGULAR_PATH = "./RegularGraphs/"
OUTPUT_PATH = "./GraphData/"
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Create label mapping for categories
unique_categories = name_grouped['ICD11Code'].unique()
category_to_label = {category: idx for idx, category in enumerate(unique_categories)}
label_to_category = {v: k for k, v in category_to_label.items()}

# Initialize files
node_connections = []
graph_indicator = []
indicator_to_common_name = {}
graph_labels = []
graph_label_to_category = label_to_category

# Track graph index
graph_idx = 0
print(category_to_label)
print(label_to_category)
print(category_to_label.keys())

# Loop through all categories and files in REGULAR_PATH
for category in os.listdir(REGULAR_PATH):
    category_path = os.path.join(REGULAR_PATH, category)

    if os.path.isdir(category_path):  # Check if it's a folder
        # Filter to get matching rows in name_grouped for the category
        category_rows = name_grouped[name_grouped['ICD11Code'] == category]

        # Loop through each file in the category
        for file_name in os.listdir(category_path):
            if file_name.endswith(".npz"):
                # Load the regular graph
                graph_path = os.path.join(category_path, file_name)
                adjacency_matrix = load_npz(graph_path)

                # Convert adjacency matrix to NetworkX graph
                G = nx.from_scipy_sparse_matrix(adjacency_matrix)
                if G.number_of_edges() > 0:
                    # Add node connections
                    for u, v in G.edges():
                        node_connections.append(f"{u}, {v}")
                        node_connections.append(f"{v}, {u}")
                    
                    # Add graph indicator for all nodes in this graph
                    graph_indicator.extend([graph_idx + 1] * G.number_of_edges() * 2)
                    # Map graph index to common name
                    common_name = file_name.replace(".npz", "")
                    indicator_to_common_name[graph_idx + 1] = common_name

                    # Add graph label
                    category_label = category_to_label[category]
                    graph_labels.append(category_label)

                    # Update graph and node offset
                    graph_idx += 1

# Save all 5 files
# 1. Graph Node Connections
with open(os.path.join(OUTPUT_PATH, "graph_A.txt"), "w") as f:
    f.write("\n".join(node_connections))
print("✅ graph_edges.txt saved!")

# 2. Graph Indicator
with open(os.path.join(OUTPUT_PATH, "graph_graph_indicator.txt"), "w") as f:
    f.write("\n".join(map(str, graph_indicator)))
print("✅ graph_indicator.txt saved!")

# 3. Indicator to Common Name
with open(os.path.join(OUTPUT_PATH, "indicator_to_common_name.txt"), "w") as f:
    for idx, name in indicator_to_common_name.items():
        f.write(f"{idx}: {name}\n")
print("✅ indicator_to_common_name.txt saved!")

# 4. Graph Labels
with open(os.path.join(OUTPUT_PATH, "graph_graph_labels.txt"), "w") as f:
    f.write("\n".join(map(str, graph_labels)))
print("✅ graph_labels.txt saved!")

# 5. Graph Label to Category
with open(os.path.join(OUTPUT_PATH, "graph_label_to_category.txt"), "w") as f:
    for label, category in graph_label_to_category.items():
        f.write(f"{label}: {category}\n")
print("✅ graph_label_to_category.txt saved!")


{'5': 0, '11': 1, 'X': 2, '16': 3, '6': 4, '14': 5, '2': 6, '12': 7, '1': 8, '8': 9, '4': 10, '3': 11, '10': 12, '13': 13, '19': 14, '9': 15, '7': 16, '21': 17, '18': 18, '15': 19, '20': 20, '22': 21, '24': 22, '23': 23, '17': 24}
{0: '5', 1: '11', 2: 'X', 3: '16', 4: '6', 5: '14', 6: '2', 7: '12', 8: '1', 9: '8', 10: '4', 11: '3', 12: '10', 13: '13', 14: '19', 15: '9', 16: '7', 17: '21', 18: '18', 19: '15', 20: '20', 21: '22', 22: '24', 23: '23', 24: '17'}
dict_keys(['5', '11', 'X', '16', '6', '14', '2', '12', '1', '8', '4', '3', '10', '13', '19', '9', '7', '21', '18', '15', '20', '22', '24', '23', '17'])
✅ graph_edges.txt saved!
✅ graph_indicator.txt saved!
✅ indicator_to_common_name.txt saved!
✅ graph_labels.txt saved!
✅ graph_label_to_category.txt saved!


In [20]:
import os
import numpy as np
import pandas as pd
import networkx as nx
from scipy.sparse import load_npz

# Paths
REGULAR_PATH = "./RegularGraphs/"

# === CONFIGURATION ===
# Choose category and graph to process
SELECTED_CATEGORY = "6"  # Example: "Graphs_A"
DISEASE_NAME = "Bipolar affective disorder, currently depressed, mild"
SELECTED_GRAPH = DISEASE_NAME + ".npz"   # Example: "example_graph.npz"
OUTPUT_PATH = "./SINGLE/" + DISEASE_NAME
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Create label mapping for categories
unique_categories = name_grouped['ICD11Code'].unique()
category_to_label = {category: idx for idx, category in enumerate(unique_categories)}
label_to_category = {v: k for k, v in category_to_label.items()}

# Initialize files
node_connections = []
graph_indicator = []
indicator_to_common_name = {}
graph_labels = []
graph_label_to_category = label_to_category

# Track graph index
graph_idx = 0
print(category_to_label)
print(label_to_category)

# Process only the selected graph
category_path = os.path.join(REGULAR_PATH, SELECTED_CATEGORY)
graph_path = os.path.join(category_path, SELECTED_GRAPH)

# Check if the selected graph exists
if os.path.isfile(graph_path) and SELECTED_GRAPH.endswith(".npz"):
    # Load the selected graph
    adjacency_matrix = load_npz(graph_path)

    # Convert adjacency matrix to NetworkX graph
    G = nx.from_scipy_sparse_matrix(adjacency_matrix)

    # Add node connections
    for u, v in G.edges():
        node_connections.append(f"{u}, {v}")
        node_connections.append(f"{v}, {u}")

    # Add graph indicator for all nodes in this graph
    graph_indicator.extend([graph_idx + 1] * G.number_of_edges() * 2)
    
    # Map graph index to common name
    common_name = SELECTED_GRAPH.replace(".npz", "")
    indicator_to_common_name[graph_idx + 1] = common_name

    # Add graph label
    category_label = category_to_label[SELECTED_CATEGORY]
    graph_labels.append(category_label)

    # Save all 5 files
    # 1. Graph Node Connections
    with open(os.path.join(OUTPUT_PATH, DISEASE_NAME + "_A.txt"), "w") as f:
        f.write("\n".join(node_connections))
    print("✅ graph_A.txt saved!")

    # 2. Graph Indicator
    with open(os.path.join(OUTPUT_PATH, DISEASE_NAME + "_graph_indicator.txt"), "w") as f:
        f.write("\n".join(map(str, graph_indicator)))
    print("✅ graph_graph_indicator.txt saved!")

    # 3. Indicator to Common Name
    with open(os.path.join(OUTPUT_PATH, DISEASE_NAME + "_indicator_to_common_name.txt"), "w") as f:
        for idx, name in indicator_to_common_name.items():
            f.write(f"{idx}: {name}\n")
    print("✅ indicator_to_common_name.txt saved!")

    # 4. Graph Labels
    with open(os.path.join(OUTPUT_PATH, DISEASE_NAME + "_graph_labels.txt"), "w") as f:
        f.write("\n".join(map(str, graph_labels)))
    print("✅ graph_graph_labels.txt saved!")

    # 5. Graph Label to Category
    with open(os.path.join(OUTPUT_PATH, DISEASE_NAME + "_label_to_category.txt"), "w") as f:
        for label, category in graph_label_to_category.items():
            f.write(f"{label}: {category}\n")
    print("✅ graph_label_to_category.txt saved!")

else:
    print(f"❌ Graph '{SELECTED_GRAPH}' not found in category '{SELECTED_CATEGORY}'!")


{'5': 0, '11': 1, 'X': 2, '16': 3, '6': 4, '14': 5, '2': 6, '12': 7, '1': 8, '8': 9, '4': 10, '3': 11, '10': 12, '13': 13, '19': 14, '9': 15, '7': 16, '21': 17, '18': 18, '15': 19, '20': 20, '22': 21, '24': 22, '23': 23, '17': 24}
{0: '5', 1: '11', 2: 'X', 3: '16', 4: '6', 5: '14', 6: '2', 7: '12', 8: '1', 9: '8', 10: '4', 11: '3', 12: '10', 13: '13', 14: '19', 15: '9', 16: '7', 17: '21', 18: '18', 19: '15', 20: '20', 21: '22', 22: '24', 23: '23', 24: '17'}
✅ graph_A.txt saved!
✅ graph_graph_indicator.txt saved!
✅ indicator_to_common_name.txt saved!
✅ graph_graph_labels.txt saved!
✅ graph_label_to_category.txt saved!
